# Feature engineering | Sebislaw

## Libraries

In [1]:
from os.path  import join
import random
import itertools
import math

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression, LassoCV, LogisticRegression
from sklearn.model_selection import train_test_split, TimeSeriesSplit
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import brier_score_loss
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV

## Data

In [2]:
data_path = '..\\..\\data'

# The Basics ------------------------------------------------------------------------
# Men
MTeams = pd.read_csv(join(data_path, 'MTeams.csv'))
MSeasons = pd.read_csv(join(data_path, 'MSeasons.csv'))
MNCAATourneySeeds = pd.read_csv(join(data_path, 'MNCAATourneySeeds.csv'))
MRegularSeasonCompactResults = pd.read_csv(join(data_path, 'MRegularSeasonCompactResults.csv'))
MNCAATourneyCompactResults = pd.read_csv(join(data_path, 'MNCAATourneyCompactResults.csv'))
# Women
WTeams = pd.read_csv(join(data_path, 'WTeams.csv'))
WSeasons = pd.read_csv(join(data_path, 'WSeasons.csv'))
WNCAATourneySeeds = pd.read_csv(join(data_path, 'WNCAATourneySeeds.csv'))
WRegularSeasonCompactResults = pd.read_csv(join(data_path, 'WRegularSeasonCompactResults.csv'))
WNCAATourneyCompactResults = pd.read_csv(join(data_path, 'WNCAATourneyCompactResults.csv'))
# Other
SampleSubmissionStage1 = pd.read_csv(join(data_path, 'SampleSubmissionStage1.csv'))
SampleSubmissionStage2 = pd.read_csv(join(data_path, 'SampleSubmissionStage2.csv'))
SeedBenchmarkStage1 = pd.read_csv(join(data_path, 'SeedBenchmarkStage1.csv'))

# Team Box Scores ------------------------------------------------------------------------
# Men
MRegularSeasonDetailedResults = pd.read_csv(join(data_path, 'MRegularSeasonDetailedResults.csv'))
MNCAATourneyDetailedResults = pd.read_csv(join(data_path, 'MNCAATourneyDetailedResults.csv'))
# Women
WRegularSeasonDetailedResults = pd.read_csv(join(data_path, 'WRegularSeasonDetailedResults.csv'))
WNCAATourneyDetailedResults = pd.read_csv(join(data_path, 'WNCAATourneyDetailedResults.csv'))

# Geography ------------------------------------------------------------------------
# All
Cities = pd.read_csv(join(data_path, 'Cities.csv'))
Conferences = pd.read_csv(join(data_path, 'Conferences.csv'))
# Men
MGameCities = pd.read_csv(join(data_path, 'MGameCities.csv'))
# Women
WGameCities = pd.read_csv(join(data_path, 'WGameCities.csv'))

# Public Rankings ------------------------------------------------------------------------
# Men
MMasseyOrdinals = pd.read_csv(join(data_path, 'MMasseyOrdinals.csv')) # men only

# Supplements ------------------------------------------------------------------------
# Men
MTeamCoaches = pd.read_csv(join(data_path, 'MTeamCoaches.csv')) # men only
MTeamConferences = pd.read_csv(join(data_path, 'MTeamConferences.csv'))
MConferenceTourneyGames = pd.read_csv(join(data_path, 'MConferenceTourneyGames.csv'))
MSecondaryTourneyTeams = pd.read_csv(join(data_path, 'MSecondaryTourneyTeams.csv'))
MSecondaryTourneyCompactResults = pd.read_csv(join(data_path, 'MSecondaryTourneyCompactResults.csv'))
MTeamSpellings = pd.read_csv(join(data_path, "MTeamSpellings.csv"), encoding='cp1252')
MNCAATourneySlots = pd.read_csv(join(data_path, 'MNCAATourneySlots.csv'))
MNCAATourneySeedRoundSlots = pd.read_csv(join(data_path, 'MNCAATourneySeedRoundSlots.csv')) # men only
# Women
WTeamConferences = pd.read_csv(join(data_path, 'WTeamConferences.csv'))
WConferenceTourneyGames = pd.read_csv(join(data_path, 'WConferenceTourneyGames.csv'))
WSecondaryTourneyTeams = pd.read_csv(join(data_path, 'WSecondaryTourneyTeams.csv'))
WSecondaryTourneyCompactResults = pd.read_csv(join(data_path, 'WSecondaryTourneyCompactResults.csv'))
WTeamSpellings = pd.read_csv(join(data_path, 'WTeamSpellings.csv'), encoding='cp1252')
WNCAATourneySlots = pd.read_csv(join(data_path, 'WNCAATourneySlots.csv'))

## Analysing data

### What data do we have from regular season?

In [3]:
df = MNCAATourneyDetailedResults.copy()
for l in list(df):
    print(l, end=', ')
# list(df)

df[['Season', 'DayNum', 'NumOT',
 'WTeamID',  'WScore', 'WLoc',
 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF',
 'LTeamID', 'LScore',
 'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF'
]].head()

Season, DayNum, WTeamID, WScore, LTeamID, LScore, WLoc, NumOT, WFGM, WFGA, WFGM3, WFGA3, WFTM, WFTA, WOR, WDR, WAst, WTO, WStl, WBlk, WPF, LFGM, LFGA, LFGM3, LFGA3, LFTM, LFTA, LOR, LDR, LAst, LTO, LStl, LBlk, LPF, 

,Season,DayNum,NumOT,WTeamID,WScore,WLoc,WFGM,WFGA,WFGM3,WFGA3,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,134,1,1421,92,N,32,69,11,29,...,31,14,31,17,28,16,15,5,0,22
1,2003,136,0,1112,80,N,31,66,7,23,...,16,7,7,8,26,12,17,10,3,15
2,2003,136,0,1113,84,N,31,59,6,14,...,28,14,21,20,22,11,12,2,5,18
3,2003,136,0,1141,79,N,29,53,3,7,...,17,12,17,14,17,20,21,6,6,21
4,2003,136,1,1143,76,N,27,64,7,20,...,21,15,20,10,26,16,14,5,8,19


### Prepare data frame
The following code (slightly modified) in "Prepare data frame" is from \
https://www.kaggle.com/code/rustyb/paris-madness-2023#Time-to-build-some-models!

In [4]:
def prepare_data(df):
    
    """
    This function duplicates and flips a game record.
    Now two records with the same data are present, 
    but viewed from perspectives of two different teams.
    """

    df = df[[
     'Season', 'DayNum', 'NumOT',
     'WTeamID',  'WScore', 'WLoc',
     'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF',
     'LTeamID', 'LScore',
     'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF'
    ]]
    dfswap = df[[
     'Season', 'DayNum', 'NumOT',
     'LTeamID', 'LScore', 'WLoc',
     'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF',
     'WTeamID',  'WScore',
     'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF',
    ]].copy()

    dfswap.loc[df['WLoc'] == 'H', 'WLoc'] = 'A'
    dfswap.loc[df['WLoc'] == 'A', 'WLoc'] = 'H'
    
    df = df.rename(columns={'WLoc': 'location'})
    dfswap = dfswap.rename(columns={'WLoc': 'location'})
    
    df.columns = [x.replace('W','T1_').replace('L','T2_') for x in list(df.columns)]
    dfswap.columns = [x.replace('L','T1_').replace('W','T2_') for x in list(dfswap.columns)]

    output = pd.concat([df, dfswap]).reset_index(drop=True)
    output.loc[output.location=='N','location'] = '0'
    output.loc[output.location=='H','location'] = '1'
    output.loc[output.location=='A','location'] = '-1'
    output.location = output.location.astype(int)
    
    output['PointDiff'] = output['T1_Score'] - output['T2_Score']

    return output

In [5]:
# read data frames 
regular_results = MRegularSeasonDetailedResults.copy()
tourney_results = MNCAATourneyDetailedResults.copy()
seeds = MNCAATourneySeeds.copy()

In [6]:
# make data frames with extra rows to represent the perspective of losing team
regular_data = prepare_data(regular_results)
tourney_data  = prepare_data(tourney_results)

regular_data.head()

,Season,DayNum,NumOT,T1_TeamID,T1_Score,location,T1_FGM,T1_FGA,T1_FGM3,T1_FGA3,...,T2_FTM,T2_FTA,T2_OR,T2_DR,T2_Ast,T2_TO,T2_Stl,T2_Blk,T2_PF,PointDiff
0,2003,10,0,1104,68,0,27,58,3,14,...,16,22,10,22,8,18,9,2,20,6
1,2003,10,0,1272,70,0,26,62,8,20,...,9,20,20,25,7,12,8,6,16,7
2,2003,11,0,1266,73,0,24,58,8,18,...,14,23,31,22,9,12,2,5,23,12
3,2003,11,0,1296,56,0,18,38,3,9,...,8,15,17,20,9,19,4,3,23,6
4,2003,11,0,1400,77,0,30,61,6,14,...,17,27,21,15,12,10,7,1,14,6


In [7]:
# data frame with mean game statistics for a given team in a given season
season_statistics  = regular_data.groupby(["Season", 'T1_TeamID'])[
    [
        'T1_FGM','T1_FGA','T1_FGM3','T1_FGA3','T1_FTM','T1_FTA','T1_OR','T1_DR','T1_Ast','T1_TO','T1_Stl','T1_Blk','T1_PF',
        'T2_FGM','T2_FGA','T2_FGM3','T2_FGA3','T2_FTM','T2_FTA','T2_OR','T2_DR','T2_Ast','T2_TO','T2_Stl','T2_Blk','T2_PF',
        'PointDiff'
    ]
].agg('mean').reset_index()
season_statistics.head()

,Season,T1_TeamID,T1_FGM,T1_FGA,T1_FGM3,T1_FGA3,T1_FTM,T1_FTA,T1_OR,T1_DR,...,T2_FTM,T2_FTA,T2_OR,T2_DR,T2_Ast,T2_TO,T2_Stl,T2_Blk,T2_PF,PointDiff
0,2003,1102,19.142857,39.785714,7.821429,20.821429,11.142857,17.107143,4.178571,16.821429,...,13.678571,19.250000,9.607143,20.142857,9.142857,12.964286,5.428571,1.571429,18.357143,0.250000
1,2003,1103,27.148148,55.851852,5.444444,16.074074,19.037037,25.851852,9.777778,19.925926,...,15.925926,22.148148,12.037037,22.037037,15.481481,15.333333,6.407407,2.851852,22.444444,0.629630
2,2003,1104,24.035714,57.178571,6.357143,19.857143,14.857143,20.928571,13.571429,23.928571,...,12.142857,17.142857,10.892857,22.642857,11.678571,13.857143,5.535714,3.178571,19.250000,4.285714
3,2003,1105,24.384615,61.615385,7.576923,20.769231,15.423077,21.846154,13.500000,23.115385,...,16.384615,24.500000,13.192308,26.384615,15.807692,18.807692,9.384615,4.192308,19.076923,-4.884615
4,2003,1106,23.428571,55.285714,6.107143,17.642857,10.642857,16.464286,12.285714,23.857143,...,15.535714,21.964286,11.321429,22.357143,11.785714,15.071429,8.785714,3.178571,16.142857,-0.142857


In [8]:
# mean statistics for team and team's opponent's
season_statistics_T1 = season_statistics .copy()
season_statistics_T2 = season_statistics .copy()

season_statistics_T1.columns = ["T1_" + x.replace("T1_","").replace("T2_","opponent_") for x in list(season_statistics_T1.columns)]
season_statistics_T2.columns = ["T2_" + x.replace("T1_","").replace("T2_","opponent_") for x in list(season_statistics_T2.columns)]
season_statistics_T1.columns.values[0] = "Season"
season_statistics_T2.columns.values[0] = "Season"

season_statistics_T1.head()

,Season,T1_TeamID,T1_FGM,T1_FGA,T1_FGM3,T1_FGA3,T1_FTM,T1_FTA,T1_OR,T1_DR,...,T1_opponent_FTM,T1_opponent_FTA,T1_opponent_OR,T1_opponent_DR,T1_opponent_Ast,T1_opponent_TO,T1_opponent_Stl,T1_opponent_Blk,T1_opponent_PF,T1_PointDiff
0,2003,1102,19.142857,39.785714,7.821429,20.821429,11.142857,17.107143,4.178571,16.821429,...,13.678571,19.250000,9.607143,20.142857,9.142857,12.964286,5.428571,1.571429,18.357143,0.250000
1,2003,1103,27.148148,55.851852,5.444444,16.074074,19.037037,25.851852,9.777778,19.925926,...,15.925926,22.148148,12.037037,22.037037,15.481481,15.333333,6.407407,2.851852,22.444444,0.629630
2,2003,1104,24.035714,57.178571,6.357143,19.857143,14.857143,20.928571,13.571429,23.928571,...,12.142857,17.142857,10.892857,22.642857,11.678571,13.857143,5.535714,3.178571,19.250000,4.285714
3,2003,1105,24.384615,61.615385,7.576923,20.769231,15.423077,21.846154,13.500000,23.115385,...,16.384615,24.500000,13.192308,26.384615,15.807692,18.807692,9.384615,4.192308,19.076923,-4.884615
4,2003,1106,23.428571,55.285714,6.107143,17.642857,10.642857,16.464286,12.285714,23.857143,...,15.535714,21.964286,11.321429,22.357143,11.785714,15.071429,8.785714,3.178571,16.142857,-0.142857


In [9]:
# data frame containing game's result
tourney_data = tourney_data[['Season', 'DayNum', 'T1_TeamID', 'T1_Score', 'T2_TeamID' ,'T2_Score', 'location']]
tourney_data = pd.merge(tourney_data, season_statistics_T1, on = ['Season', 'T1_TeamID'], how = 'left')
tourney_data = pd.merge(tourney_data, season_statistics_T2, on = ['Season', 'T2_TeamID'], how = 'left')

tourney_data.head()

,Season,DayNum,T1_TeamID,T1_Score,T2_TeamID,T2_Score,location,T1_FGM,T1_FGA,T1_FGM3,...,T2_opponent_FTM,T2_opponent_FTA,T2_opponent_OR,T2_opponent_DR,T2_opponent_Ast,T2_opponent_TO,T2_opponent_Stl,T2_opponent_Blk,T2_opponent_PF,T2_PointDiff
0,2003,134,1421,92,1411,84,0,24.379310,56.793103,6.482759,...,11.966667,18.733333,11.933333,22.866667,13.766667,14.333333,8.000000,2.600000,21.633333,1.966667
1,2003,136,1112,80,1436,51,0,30.321429,65.714286,7.035714,...,10.551724,15.758621,9.586207,21.862069,13.275862,13.000000,7.103448,3.655172,17.931034,4.655172
2,2003,136,1113,84,1272,71,0,27.206897,56.896552,4.000000,...,13.379310,20.724138,12.344828,23.586207,13.310345,15.068966,7.275862,3.172414,19.931034,8.689655
3,2003,136,1141,79,1166,73,0,26.620690,52.689655,6.827586,...,11.696970,16.696970,11.060606,21.363636,12.363636,17.060606,6.333333,2.575758,19.393939,14.909091
4,2003,136,1143,76,1301,74,0,27.344828,58.724138,6.413793,...,15.400000,21.200000,10.533333,21.433333,12.566667,14.633333,7.433333,2.833333,19.333333,4.400000


In [10]:
# data frame with win fraction from last x days for a given team in a given season
last14days_stats_T1 = regular_data.loc[regular_data.DayNum>118].reset_index(drop=True)
last14days_stats_T1['win'] = np.where(last14days_stats_T1['PointDiff']>0,1,0)
last14days_stats_T1 = last14days_stats_T1.groupby(['Season','T1_TeamID'])['win'].mean().reset_index(name='T1_win_ratio_14d')

last14days_stats_T2 = regular_data.loc[regular_data.DayNum>118].reset_index(drop=True)
last14days_stats_T2['win'] = np.where(last14days_stats_T2['PointDiff']<0,1,0)
last14days_stats_T2 = last14days_stats_T2.groupby(['Season','T2_TeamID'])['win'].mean().reset_index(name='T2_win_ratio_14d')

last14days_stats_T1.head()

,Season,T1_TeamID,T1_win_ratio_14d
0,2003,1102,0.333333
1,2003,1103,0.333333
2,2003,1104,0.333333
3,2003,1105,0.000000
4,2003,1106,0.500000


In [11]:
# add to tourney_data column with win fraction for winning and losing team
tourney_data = pd.merge(tourney_data, last14days_stats_T1, on = ['Season', 'T1_TeamID'], how = 'left')
tourney_data = pd.merge(tourney_data, last14days_stats_T2, on = ['Season', 'T2_TeamID'], how = 'left')

tourney_data.head()

,Season,DayNum,T1_TeamID,T1_Score,T2_TeamID,T2_Score,location,T1_FGM,T1_FGA,T1_FGM3,...,T2_opponent_OR,T2_opponent_DR,T2_opponent_Ast,T2_opponent_TO,T2_opponent_Stl,T2_opponent_Blk,T2_opponent_PF,T2_PointDiff,T1_win_ratio_14d,T2_win_ratio_14d
0,2003,134,1421,92,1411,84,0,24.379310,56.793103,6.482759,...,11.933333,22.866667,13.766667,14.333333,8.000000,2.600000,21.633333,1.966667,1.000000,0.833333
1,2003,136,1112,80,1436,51,0,30.321429,65.714286,7.035714,...,9.586207,21.862069,13.275862,13.000000,7.103448,3.655172,17.931034,4.655172,0.666667,1.000000
2,2003,136,1113,84,1272,71,0,27.206897,56.896552,4.000000,...,12.344828,23.586207,13.310345,15.068966,7.275862,3.172414,19.931034,8.689655,0.666667,0.750000
3,2003,136,1141,79,1166,73,0,26.620690,52.689655,6.827586,...,11.060606,21.363636,12.363636,17.060606,6.333333,2.575758,19.393939,14.909091,1.000000,1.000000
4,2003,136,1143,76,1301,74,0,27.344828,58.724138,6.413793,...,10.533333,21.433333,12.566667,14.633333,7.433333,2.833333,19.333333,4.400000,0.333333,0.600000


In [12]:
# a win label for team's matchups
regular_season_effects = regular_data[['Season','T1_TeamID','T2_TeamID','PointDiff']].copy()
regular_season_effects['T1_TeamID'] = regular_season_effects['T1_TeamID'].astype(str)
regular_season_effects['T2_TeamID'] = regular_season_effects['T2_TeamID'].astype(str)
regular_season_effects['win'] = np.where(regular_season_effects['PointDiff']>0,1,0)

march_madness = pd.merge(seeds[['Season','TeamID']],seeds[['Season','TeamID']],on='Season')
march_madness.columns = ['Season', 'T1_TeamID', 'T2_TeamID']
march_madness.T1_TeamID = march_madness.T1_TeamID.astype(str)
march_madness.T2_TeamID = march_madness.T2_TeamID.astype(str)

regular_season_effects = pd.merge(regular_season_effects, march_madness, on = ['Season','T1_TeamID','T2_TeamID'])

regular_season_effects.head()

,Season,T1_TeamID,T2_TeamID,PointDiff,win
0,2003,1104,1328,6,1
1,2003,1272,1393,7,1
2,2003,1323,1237,44,1
3,2003,1242,1221,24,1
4,2003,1390,1462,1,1


In [13]:
# get seeds with no regional division
seeds['seed'] = seeds['Seed'].apply(lambda x: int(x[1:3]))

seeds.head()

,Season,Seed,TeamID,seed
0,1985,W01,1207,1
1,1985,W02,1210,2
2,1985,W03,1228,3
3,1985,W04,1260,4
4,1985,W05,1374,5


In [14]:
# give each team a raw seed
seeds_T1 = seeds[['Season','TeamID','seed']].copy()
seeds_T2 = seeds[['Season','TeamID','seed']].copy()
seeds_T1.columns = ['Season','T1_TeamID','T1_seed']
seeds_T2.columns = ['Season','T2_TeamID','T2_seed']

seeds_T1.head()

,Season,T1_TeamID,T1_seed
0,1985,1207,1
1,1985,1210,2
2,1985,1228,3
3,1985,1260,4
4,1985,1374,5


In [15]:
# add seeds to turney data for team 1 and team 2
tourney_data = pd.merge(tourney_data, seeds_T1, on = ['Season', 'T1_TeamID'], how = 'left')
tourney_data = pd.merge(tourney_data, seeds_T2, on = ['Season', 'T2_TeamID'], how = 'left')

tourney_data.head()

,Season,DayNum,T1_TeamID,T1_Score,T2_TeamID,T2_Score,location,T1_FGM,T1_FGA,T1_FGM3,...,T2_opponent_Ast,T2_opponent_TO,T2_opponent_Stl,T2_opponent_Blk,T2_opponent_PF,T2_PointDiff,T1_win_ratio_14d,T2_win_ratio_14d,T1_seed,T2_seed
0,2003,134,1421,92,1411,84,0,24.379310,56.793103,6.482759,...,13.766667,14.333333,8.000000,2.600000,21.633333,1.966667,1.000000,0.833333,16,16
1,2003,136,1112,80,1436,51,0,30.321429,65.714286,7.035714,...,13.275862,13.000000,7.103448,3.655172,17.931034,4.655172,0.666667,1.000000,1,16
2,2003,136,1113,84,1272,71,0,27.206897,56.896552,4.000000,...,13.310345,15.068966,7.275862,3.172414,19.931034,8.689655,0.666667,0.750000,10,7
3,2003,136,1141,79,1166,73,0,26.620690,52.689655,6.827586,...,12.363636,17.060606,6.333333,2.575758,19.393939,14.909091,1.000000,1.000000,11,6
4,2003,136,1143,76,1301,74,0,27.344828,58.724138,6.413793,...,12.566667,14.633333,7.433333,2.833333,19.333333,4.400000,0.333333,0.600000,8,9


In [16]:
# add a seed difference column
tourney_data["Seed_diff"] = tourney_data["T1_seed"] - tourney_data["T2_seed"]

tourney_data

,Season,DayNum,T1_TeamID,T1_Score,T2_TeamID,T2_Score,location,T1_FGM,T1_FGA,T1_FGM3,...,T2_opponent_TO,T2_opponent_Stl,T2_opponent_Blk,T2_opponent_PF,T2_PointDiff,T1_win_ratio_14d,T2_win_ratio_14d,T1_seed,T2_seed,Seed_diff
0,2003,134,1421,92,1411,84,0,24.379310,56.793103,6.482759,...,14.333333,8.000000,2.600000,21.633333,1.966667,1.000000,0.833333,16,16,0
1,2003,136,1112,80,1436,51,0,30.321429,65.714286,7.035714,...,13.000000,7.103448,3.655172,17.931034,4.655172,0.666667,1.000000,1,16,-15
2,2003,136,1113,84,1272,71,0,27.206897,56.896552,4.000000,...,15.068966,7.275862,3.172414,19.931034,8.689655,0.666667,0.750000,10,7,3
3,2003,136,1141,79,1166,73,0,26.620690,52.689655,6.827586,...,17.060606,6.333333,2.575758,19.393939,14.909091,1.000000,1.000000,11,6,5
4,2003,136,1143,76,1301,74,0,27.344828,58.724138,6.413793,...,14.633333,7.433333,2.833333,19.333333,4.400000,0.333333,0.600000,8,9,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2759,2024,146,1181,64,1301,76,0,28.531250,59.218750,8.343750,...,11.555556,5.194444,3.111111,16.194444,3.666667,0.333333,0.714286,4,11,-7
2760,2024,146,1397,66,1345,72,0,27.437500,61.750000,8.718750,...,9.515152,6.848485,2.272727,20.666667,13.242424,0.333333,0.750000,2,1,1
2761,2024,152,1104,72,1163,86,0,30.906250,64.750000,11.062500,...,10.088235,5.500000,2.882353,16.852941,17.058824,0.333333,1.000000,4,1,3
2762,2024,152,1301,50,1345,63,0,27.388889,60.972222,7.055556,...,9.515152,6.848485,2.272727,20.666667,13.242424,0.714286,0.750000,11,1,10


In [17]:
tourney_data

,Season,DayNum,T1_TeamID,T1_Score,T2_TeamID,T2_Score,location,T1_FGM,T1_FGA,T1_FGM3,...,T2_opponent_TO,T2_opponent_Stl,T2_opponent_Blk,T2_opponent_PF,T2_PointDiff,T1_win_ratio_14d,T2_win_ratio_14d,T1_seed,T2_seed,Seed_diff
0,2003,134,1421,92,1411,84,0,24.379310,56.793103,6.482759,...,14.333333,8.000000,2.600000,21.633333,1.966667,1.000000,0.833333,16,16,0
1,2003,136,1112,80,1436,51,0,30.321429,65.714286,7.035714,...,13.000000,7.103448,3.655172,17.931034,4.655172,0.666667,1.000000,1,16,-15
2,2003,136,1113,84,1272,71,0,27.206897,56.896552,4.000000,...,15.068966,7.275862,3.172414,19.931034,8.689655,0.666667,0.750000,10,7,3
3,2003,136,1141,79,1166,73,0,26.620690,52.689655,6.827586,...,17.060606,6.333333,2.575758,19.393939,14.909091,1.000000,1.000000,11,6,5
4,2003,136,1143,76,1301,74,0,27.344828,58.724138,6.413793,...,14.633333,7.433333,2.833333,19.333333,4.400000,0.333333,0.600000,8,9,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2759,2024,146,1181,64,1301,76,0,28.531250,59.218750,8.343750,...,11.555556,5.194444,3.111111,16.194444,3.666667,0.333333,0.714286,4,11,-7
2760,2024,146,1397,66,1345,72,0,27.437500,61.750000,8.718750,...,9.515152,6.848485,2.272727,20.666667,13.242424,0.333333,0.750000,2,1,1
2761,2024,152,1104,72,1163,86,0,30.906250,64.750000,11.062500,...,10.088235,5.500000,2.882353,16.852941,17.058824,0.333333,1.000000,4,1,3
2762,2024,152,1301,50,1345,63,0,27.388889,60.972222,7.055556,...,9.515152,6.848485,2.272727,20.666667,13.242424,0.714286,0.750000,11,1,10


In [18]:
# a vector with game's point difference between team 1 and team 2
y_points = tourney_data['T1_Score'] - tourney_data['T2_Score']
y_points.head()

0     8
1    29
2    13
3     6
4     2
dtype: int64

Now the resulting tourney_data has
- which teams were facing in a tournament game and what was the outcome
- what were the mean statistics in regular season for each team, as well as the mean statistics for opponent of each team
- what were the seeds and how much was the seed difference

Ideas to modify:
- change how many days we look back when calculating win fraction,
- change how many seasons (or even days) we look back when calculating mean statistics,
- include tuornament statistics in mean statistics for a team (if we decide to look back more than just the current season),
- if we look at statistics from regular season include the information about the location, as this could impact all other statistics,
- include a feature with team quality,
- don't include a certain (2020) season.

What's next?
- select which statistics to include,
- give certain mathcups (like seed 1 vs 16) a probability of winning of 1 etc,
- we will know which teams qualiffy into the tournament on march 16.
- how do we calculate teams' quality? On average team's performance changes quickly. The mean autocorrelation of fraction of win for teams is only 0.25 1 season back, 0.1 2 season back and from 3 seasons back onwards it is <=0. So we should only look at the current season's games when calculating team's performance.

## Data preparation pipeline

In [19]:
def get_data(regular_results, tourney_results, seeds):

    def prepare_data(df):
        
        """
        This function duplicates and flips a game record.
        Now two records with the same data are present, 
        but viewed from perspectives of two different teams.
        """
    
        df = df[[
         'Season', 'DayNum', 'NumOT',
         'WTeamID',  'WScore', 'WLoc',
         'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF',
         'LTeamID', 'LScore',
         'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF'
        ]]
        dfswap = df[[
         'Season', 'DayNum', 'NumOT',
         'LTeamID', 'LScore', 'WLoc',
         'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF',
         'WTeamID',  'WScore',
         'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF',
        ]].copy()
    
        dfswap.loc[df['WLoc'] == 'H', 'WLoc'] = 'A'
        dfswap.loc[df['WLoc'] == 'A', 'WLoc'] = 'H'
        
        df = df.rename(columns={'WLoc': 'location'})
        dfswap = dfswap.rename(columns={'WLoc': 'location'})
        
        df.columns = [x.replace('W','T1_').replace('L','T2_') for x in list(df.columns)]
        dfswap.columns = [x.replace('L','T1_').replace('W','T2_') for x in list(dfswap.columns)]
    
        output = pd.concat([df, dfswap]).reset_index(drop=True)
        output.loc[output.location=='N','location'] = '0'
        output.loc[output.location=='H','location'] = '1'
        output.loc[output.location=='A','location'] = '-1'
        output.location = output.location.astype(int)
        
        output['PointDiff'] = output['T1_Score'] - output['T2_Score']
    
        return output
    
    # make data frames with extra rows to represent the perspective of losing team
    regular_data = prepare_data(regular_results)
    tourney_data  = prepare_data(tourney_results)
    
    # data frame with mean game statistics for a given team in a given season
    season_statistics  = regular_data.groupby(["Season", 'T1_TeamID'])[
        [
            'T1_FGM','T1_FGA','T1_FGM3','T1_FGA3','T1_FTM','T1_FTA','T1_OR','T1_DR','T1_Ast','T1_TO','T1_Stl','T1_Blk','T1_PF',
            'T2_FGM','T2_FGA','T2_FGM3','T2_FGA3','T2_FTM','T2_FTA','T2_OR','T2_DR','T2_Ast','T2_TO','T2_Stl','T2_Blk','T2_PF',
            'PointDiff'
        ]
    ].agg('mean').reset_index()
    
    # mean statistics for team and team's opponent's
    season_statistics_T1 = season_statistics .copy()
    season_statistics_T2 = season_statistics .copy()
    
    season_statistics_T1.columns = ["T1_" + x.replace("T1_","").replace("T2_","opponent_") for x in list(season_statistics_T1.columns)]
    season_statistics_T2.columns = ["T2_" + x.replace("T1_","").replace("T2_","opponent_") for x in list(season_statistics_T2.columns)]
    season_statistics_T1.columns.values[0] = "Season"
    season_statistics_T2.columns.values[0] = "Season"
    
    # data frame containing game's result
    tourney_data = tourney_data[['Season', 'DayNum', 'T1_TeamID', 'T1_Score', 'T2_TeamID' ,'T2_Score', 'location']]
    tourney_data = pd.merge(tourney_data, season_statistics_T1, on = ['Season', 'T1_TeamID'], how = 'left')
    tourney_data = pd.merge(tourney_data, season_statistics_T2, on = ['Season', 'T2_TeamID'], how = 'left')
    
    # data frame with win fraction from last x days for a given team in a given season
    last14days_stats_T1 = regular_data.loc[regular_data.DayNum>118].reset_index(drop=True)
    last14days_stats_T1['win'] = np.where(last14days_stats_T1['PointDiff']>0,1,0)
    last14days_stats_T1 = last14days_stats_T1.groupby(['Season','T1_TeamID'])['win'].mean().reset_index(name='T1_win_ratio_14d')
    
    last14days_stats_T2 = regular_data.loc[regular_data.DayNum>118].reset_index(drop=True)
    last14days_stats_T2['win'] = np.where(last14days_stats_T2['PointDiff']<0,1,0)
    last14days_stats_T2 = last14days_stats_T2.groupby(['Season','T2_TeamID'])['win'].mean().reset_index(name='T2_win_ratio_14d')
    
    # add to tourney_data column with win fraction for winning and losing team
    tourney_data = pd.merge(tourney_data, last14days_stats_T1, on = ['Season', 'T1_TeamID'], how = 'left')
    tourney_data = pd.merge(tourney_data, last14days_stats_T2, on = ['Season', 'T2_TeamID'], how = 'left')
    
    # get seeds with no regional division
    seeds['seed'] = seeds['Seed'].apply(lambda x: int(x[1:3]))
    
    # give each team a raw seed
    seeds_T1 = seeds[['Season','TeamID','seed']].copy()
    seeds_T2 = seeds[['Season','TeamID','seed']].copy()
    seeds_T1.columns = ['Season','T1_TeamID','T1_seed']
    seeds_T2.columns = ['Season','T2_TeamID','T2_seed']
    
    # add seeds to turney data for team 1 and team 2
    tourney_data = pd.merge(tourney_data, seeds_T1, on = ['Season', 'T1_TeamID'], how = 'left')
    tourney_data = pd.merge(tourney_data, seeds_T2, on = ['Season', 'T2_TeamID'], how = 'left')
    
    # add a seed difference column
    tourney_data["Seed_diff"] = tourney_data["T1_seed"] - tourney_data["T2_seed"]

    return tourney_data

## Feature transformations

In [20]:
regular_results = MRegularSeasonDetailedResults.copy()
tourney_results = MNCAATourneyDetailedResults.copy()
seeds = MNCAATourneySeeds.copy()

df = get_data(regular_results, tourney_results, seeds)

X = df[list(df.columns[7:999])].values
y = np.where(df['T1_Score'] - df['T2_Score'] > 0, 1, 0)

In [21]:
df

,Season,DayNum,T1_TeamID,T1_Score,T2_TeamID,T2_Score,location,T1_FGM,T1_FGA,T1_FGM3,...,T2_opponent_TO,T2_opponent_Stl,T2_opponent_Blk,T2_opponent_PF,T2_PointDiff,T1_win_ratio_14d,T2_win_ratio_14d,T1_seed,T2_seed,Seed_diff
0,2003,134,1421,92,1411,84,0,24.379310,56.793103,6.482759,...,14.333333,8.000000,2.600000,21.633333,1.966667,1.000000,0.833333,16,16,0
1,2003,136,1112,80,1436,51,0,30.321429,65.714286,7.035714,...,13.000000,7.103448,3.655172,17.931034,4.655172,0.666667,1.000000,1,16,-15
2,2003,136,1113,84,1272,71,0,27.206897,56.896552,4.000000,...,15.068966,7.275862,3.172414,19.931034,8.689655,0.666667,0.750000,10,7,3
3,2003,136,1141,79,1166,73,0,26.620690,52.689655,6.827586,...,17.060606,6.333333,2.575758,19.393939,14.909091,1.000000,1.000000,11,6,5
4,2003,136,1143,76,1301,74,0,27.344828,58.724138,6.413793,...,14.633333,7.433333,2.833333,19.333333,4.400000,0.333333,0.600000,8,9,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2759,2024,146,1181,64,1301,76,0,28.531250,59.218750,8.343750,...,11.555556,5.194444,3.111111,16.194444,3.666667,0.333333,0.714286,4,11,-7
2760,2024,146,1397,66,1345,72,0,27.437500,61.750000,8.718750,...,9.515152,6.848485,2.272727,20.666667,13.242424,0.333333,0.750000,2,1,1
2761,2024,152,1104,72,1163,86,0,30.906250,64.750000,11.062500,...,10.088235,5.500000,2.882353,16.852941,17.058824,0.333333,1.000000,4,1,3
2762,2024,152,1301,50,1345,63,0,27.388889,60.972222,7.055556,...,9.515152,6.848485,2.272727,20.666667,13.242424,0.714286,0.750000,11,1,10


In [22]:
X

array([[ 24.37931034,  56.79310345,   6.48275862, ...,  16.        ,
         16.        ,   0.        ],
       [ 30.32142857,  65.71428571,   7.03571429, ...,   1.        ,
         16.        , -15.        ],
       [ 27.20689655,  56.89655172,   4.        , ...,  10.        ,
          7.        ,   3.        ],
       ...,
       [ 30.90625   ,  64.75      ,  11.0625    , ...,   4.        ,
          1.        ,   3.        ],
       [ 27.38888889,  60.97222222,   7.05555556, ...,  11.        ,
          1.        ,  10.        ],
       [ 28.51515152,  58.39393939,   8.33333333, ...,   1.        ,
          1.        ,   0.        ]])

In [23]:
y

array([1, 1, 1, ..., 0, 0, 0])

## First models

In [24]:
model = LogisticRegression(solver='lbfgs')
model.fit(X, y)

y_prob = model.predict_proba(X)[:, 1]

score = brier_score_loss(y, y_prob)
print("Brier score:", score)

Brier score: 0.18410971024830505


C:\Users\Sebastian\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
